In [29]:
import sys
paths = [
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/util",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/config",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/common",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/common/ds",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_intrinio",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/common/web",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_portfolio",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/test/selenium/api",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/test/selenium",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/estypes",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/esmappings",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/esindexes",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_elasticsearch/utils",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch_ingestion/helper",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch_ingestion",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services/util",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_quandl",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_snp",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services/public",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_barchart",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_redis",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/models",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_twitter",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_edgar",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_fred",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_oecd",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/common/sbt_websocket",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/util/serverless/elasticsearch/helper",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/elasticsearch/helper",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/serverless/symbol_mapping",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/scripts/aws/sbt_elasticsearch",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_cryptocurrency",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/services/user",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_cbonds",
        "/Users/tbui/workstation/Stansberry-Terminal-MkI/datafactory/vendor/sbt_brokers"
]

In [30]:
sys.path += paths
# sys.path.append("C:\\Users\\tbui\\Documents\\Stansberry-Terminal-MkI\\services\\screener_sbanalysis_api\\")

In [31]:

from pandas.io import sql as psql

In [32]:
from lib.connected_db import PgsqlExecutor

In [33]:
research_db = {
        "host": "dev-df-postgresql.csodrrohkuas.us-east-1.rds.amazonaws.com",
        "port": 5432,
        "user": "df_user",
        "credentials": "Piq914sT",
        "database": "datafactory"
      }
snpsource = {
        "host": "snp-instance-1-us-east-1b.csodrrohkuas.us-east-1.rds.amazonaws.com",
        "port": 5432,
        "user": "snpx_read",
        "credentials": "Nsp563Xz",
        "database": "xpressfeed"
      }

In [34]:
db = PgsqlExecutor(snpsource)
model_db = PgsqlExecutor(research_db)

In [7]:
item_dict = {}
sql = """

select tradingitemid, tickersymbol from v2mv_sbt_company where  
is_primary_exchange = true and exchangesymbol not ilike 'mutualfund'
    and securityname ilike 'common%stock' 
             and exchangecountry = 'USA'
"""

data = db.fetch_all_rows(sql)
for row in data.query_data:
    item_dict[row.tickersymbol.upper()] = row

In [8]:
def get_current_price(symbol):
    sql = """
    SELECT  priceclose
	    FROM public.v_price_timeseries 
	        WHERE tradingitemid = %s 
	        AND pricingdate >= now()::Date -3
            
            order by pricingdate desc limit 1
    
    """
    rv = db.fetch_one_row(sql, args=[item_dict[symbol.upper()].tradingitemid])
    if rv.row_count > 0:
        return rv.query_data.priceclose

In [9]:
get_current_price('aapl')

351.59

In [10]:
import requests

In [11]:
url = "https://api.nasdaq.com/api/calendar/earnings?date=2020-06-09"

In [12]:
response = requests.get(url)
reponse_values = response.json()['data']['rows']

In [13]:
for row in reponse_values:
    print (row)
    print(row["symbol"])

{'eps': '$0.27', 'surprise': '-3.57', 'time': 'time-not-supplied', 'symbol': 'BF.B', 'name': 'Brown Forman Corporation', 'marketCap': '$32,087,987,345', 'fiscalQuarterEnding': 'Apr/2020', 'epsForecast': '$0.28', 'noOfEsts': '6'}
BF.B
{'eps': '$0.27', 'surprise': '999', 'time': 'time-not-supplied', 'symbol': 'BF.A', 'name': 'Brown Forman Corporation', 'marketCap': '$29,128,299,643', 'fiscalQuarterEnding': 'Apr/2020', 'epsForecast': '', 'noOfEsts': '0'}
BF.A
{'eps': '($0.12)', 'surprise': '-9.09', 'time': 'time-not-supplied', 'symbol': 'CHWY', 'name': 'Chewy, Inc.', 'marketCap': '$19,901,476,751', 'fiscalQuarterEnding': 'Apr/2020', 'epsForecast': '($0.11)', 'noOfEsts': '6'}
CHWY
{'eps': '($0.53)', 'surprise': '-51.43', 'time': 'time-not-supplied', 'symbol': 'TIF', 'name': 'Tiffany & Co.', 'marketCap': '$14,742,407,424', 'fiscalQuarterEnding': 'Apr/2020', 'epsForecast': '($0.35)', 'noOfEsts': '3'}
TIF
{'eps': '($0.91)', 'surprise': '-160', 'time': 'time-not-supplied', 'symbol': 'FIVE', 'n

In [14]:
from mapper.predict_logic_mapper import MapperPredictLogic, simplejson

In [15]:
mapper_logic = MapperPredictLogic(model_db)

In [16]:
merton_model = mapper_logic.get_active_session("merton_data").logic

In [17]:
data = mapper_logic.get_dcf_data("ARGO")

print(data, list(data))

[-9.2, -1182.2, -85156103.8, -129515086.76806083, -129515086.76806083] [-9.2, -1182.2, -85156103.8, -129515086.76806083, -129515086.76806083]


In [19]:
data_adj = mapper_logic.get_dcf_data_adj("ARGO")

In [20]:
dto = mapper_logic.get_active_session('dcf_adj')

In [21]:
dto = mapper_logic.get_dto(name='dcf')
dto = mapper_logic.select(dto, where_column='name')

In [22]:
mapper_logic.predict('dcf_adj', data_adj)

-29.263216

In [23]:
mapper_logic.predict('dcf', data)

-49.052353

In [24]:
mapper_logic.predict('dcf_put', data)

1.4902306

In [25]:
mapper_logic.predict('dcf_binary_up', data)

-1

In [26]:
mapper_logic.predict('dcf_binary_down', data)

-1

In [58]:
for row in reponse_values:
    up_ = down_ = 0 
    symbol = row['symbol']
    data = mapper_logic.get_dcf_data(symbol)
    data_adj = mapper_logic.get_dcf_data_adj(symbol)
    if data:
        try:
            up_, down_ = mapper_logic.predict('dcf_binary_up', data), mapper_logic.predict('dcf_binary_down', data)
        except:
            pass
        
        up_value = mapper_logic.predict('dcf', data)
        down_value = mapper_logic.predict('dcf_put', data)
        value_adj = mapper_logic.predict('dcf_adj', data_adj)
        print(value_adj)
        merton = merton_model.get(symbol)
        if not merton:
            continue
        
        if down_ == -1 or down_value < -35:
            
            print (symbol, up_value, down_value)
            print("Merton: {} {} {}".format(merton["dd"],
                                         round(float(merton["pd"]), 2),
                                           merton["snp_rating"]))
        if up_ == 1 or up_value > 35:
            print (symbol, up_value, down_value)
            print("Merton: {} {} {}".format(merton["dd"],
                                         round(float(merton["pd"]), 2),
                                           merton["snp_rating"]))
        print (symbol, up_value, down_value, value_adj) 
        
            

4.544737
TIF -21.95116 -1.8759356 4.544737
10.113638
FIVE -12.868908 7.6252565 10.113638
6.8419914
HDS 3.175607 7.2831144 6.8419914
10.911761
VRNT 0.8183343 8.491872 10.911761
4.475191
M -8.939942 2.7355735 4.475191
5.541992
AGX -3.0153992 -28.798239 5.541992
12.667341
MOV 13.464974 15.242184 12.667341
-1.2279626
LMNR -28.209805 -30.679564 -1.2279626
19.775589
CONN -22.544954 17.71684 19.775589
-52.305912
-2.253564
UEC -44.25919 -27.211746 -2.253564
13.777606
ITI -27.706593 13.882847
Merton: 4.779738774 0.0 
ITI -27.706593 13.882847 13.777606
-5.920369
LAKE -4.088923 -5.48624 -5.920369
23.179678
MVC 3.5295596 -20.683065 23.179678
1.5470216
VNCE -24.174091 -11.884266 1.5470216
-52.305912
XELA -14.95072 -14.653384 -52.305912
-52.305912
PPIH -14.95072 -14.653384 -52.305912


In [52]:
def get_prediction_by_earning_date(year, month, date):
    url = "https://api.nasdaq.com/api/calendar/earnings?date={}-{}-{}".format(
        year, month, date
    )
    response = requests.get(url)
    reponse_values = response.json()['data']['rows']
    for row in reponse_values:
        symbol = row['symbol']
        data = mapper_logic.get_dcf_data(symbol)
        if data:
            print (symbol, mapper_logic.predict('dcf', data), mapper_logic.predict('dcf_put', data))
    

In [53]:
get_prediction_by_earning_date("2020", "06", "15")

MPAA -27.869085 -19.323515
ONCS -14.95072 -14.653384
LMB -30.411224 -16.598383
JVA -29.838713 -24.690752


In [54]:
from datetime import datetime, timedelta
now = datetime.now() 

In [55]:
# from lib.date_ext import datetime_range

# for d in datetime_range(now- timedelta(days=14), now):
#     print (d.strftime("%m-%d %A"))
#     if now.isoweekday() in (6,7):
#         print (now)
#         continue
#     url = "https://api.nasdaq.com/api/calendar/earnings?date={}".format(
#         d.strftime('%Y-%m-%d')
#     )
#     response = requests.get(url)
#     response_values = response.json()['data']['rows']
#     if not response_values:
#         continue
#     for row in response_values:
#         symbol = row['symbol']
#         data = mapper_logic.get_dcf_data(symbol)
#         if data:
#             value = mapper_logic.predict('dcf', data)
#             price = get_current_price(symbol)
#             if value < -30 and price > 20:
#                 print (symbol, value, price, d)
#                 merton = merton_model[symbol]
#                 print("Merton: {} {}".format(merton["dd"],
#                                              merton["pd"]))
    
    

In [56]:
# for d in datetime_range(now- timedelta(days=14), now + timedelta(days=2)):
#     print (d.strftime("%m-%d %A"))
#     if now.isoweekday() in (6,7):
#         print (now)
#         continue
#     url = "https://api.nasdaq.com/api/calendar/earnings?date={}".format(
#         d.strftime('%Y-%m-%d')
#     )
#     response = requests.get(url)
#     response_values = response.json()['data']['rows']
#     if not response_values:
#         continue
#     for row in response_values:
#         symbol = row['symbol']
#         data = mapper_logic.get_dcf_data(symbol)
#         if data:
#             value = mapper_logic.predict('dcf', data)
#             value2 = mapper_logic.predict('dcf_put', data)
#             value_up = mapper_logic.predict('dcf_binary_up', data)
#             value_down = mapper_logic.predict('dcf_binary_down', data)
#             price = get_current_price(symbol)
#             if value_up == 1 and (value > 20 or value2 > 40) and price > 20:
#                 print (symbol, value,value2, price, d)
#                 merton = merton_model[symbol]
#                 print("Merton: {} {}".format(merton["dd"],
#                                              round(float(merton["pd"]), 2),
#                                              merton["snp_rating"]
#                                             ))
#             if value_down ==-1 and (value < -40 or value2 < -35) and price > 20:
#                 print (symbol, value,value2, price, d)
#                 merton = merton_model[symbol]
#                 print("Merton: {} {}".format(merton["dd"],
#                                              round(float(merton["pd"]), 2),
#                                              merton["snp_rating"]))
                

In [57]:
sql = """
select distinct symbol from trade.earning where EXTRACT(month FROM report_date) in (5,6)

"""
data_symbols = test_connect.fetch_all_rows(sql)
print (data_symbols.row_count)
for row in data_symbols.query_data:
    symbol = row.symbol
    data = mapper_logic.get_dcf_data(symbol)
    if data:
        value = mapper_logic.predict('dcf', data)
        
       
        price = get_current_price(symbol)
        if merton_model.get(symbol.strip().upper()) is None:
            continue

        if value < -30 and price > 20:
            print (symbol, value, price)
            merton = merton_model.get(symbol.strip().upper())
            print("Merton: {} {} {}".format(merton["dd"],
                                         round(float(merton["pd"]), 2),
                                           merton["snp_rating"]))
        if value > 20 and price > 1:
            print (symbol, value, price)
            merton = merton_model.get(symbol.strip().upper())
            print("Merton: {} {} {}".format(merton["dd"],
                                         round(float(merton["pd"]), 2),
                                           merton["snp_rating"]))


NameError: name 'test_connect' is not defined

In [ ]:
symbol = "DD"
merton = merton_model.get(symbol)
data = mapper_logic.get_dcf_data(symbol)
if data:
    value = mapper_logic.predict('dcf', data)
    value2 = mapper_logic.predict('dcf_put', data)
    value_up = mapper_logic.predict('dcf_binary_up', data)
    value_down = mapper_logic.predict('dcf_binary_down', data)
    if value < 0 and value2 < 0:
        print("predict: ", value, value2, value_down)
    if value > 0 and value2 > 0:
        print("predict: ", value, value2, value_up)
price = get_current_price(symbol)
print(price)
print("Merton: {} {} {}".format(merton["dd"],
                             round(float(merton["pd"]), 2),
                               merton["snp_rating"]))

In [ ]:
symbol = "MSEX"
merton = merton_model.get(symbol)
data = mapper_logic.get_dcf_data(symbol)
data_adj = mapper_logic.get_dcf_data_adj(symbol)
if data:
    value = mapper_logic.predict('dcf', data)
    value2 = mapper_logic.predict('dcf_put', data)
    value_adj = mapper_logic.predict('dcf_adj', data_adj)
    value_up = mapper_logic.predict('dcf_binary_up', data)
    value_down = mapper_logic.predict('dcf_binary_down', data)
    print (value, value2, value_adj, value_down, value_up)
    if value < 0 and value2 < 0:
        print("predict: ", value, value2, value_down)
    if value > 0 and value2 > 0:
        print("predict: ", value, value2, value_up)
price = get_current_price(symbol)
print(price, value_adj)
if merton:
    print("Merton: {} {} {}".format(merton["dd"],
                                 round(float(merton["pd"]), 2),
                                   merton["snp_rating"]))

In [27]:
symbol = "DD"
merton = merton_model.get(symbol)
data = mapper_logic.get_dcf_data(symbol)
data_adj = mapper_logic.get_dcf_data_adj(symbol)
if data:
    value = mapper_logic.predict('dcf', data)
    value2 = mapper_logic.predict('dcf_put', data)
    value_adj = mapper_logic.predict('dcf_adj', data_adj)
    value_up = mapper_logic.predict('dcf_binary_up', data)
    value_down = mapper_logic.predict('dcf_binary_down', data)
    print (value, value2, value_adj, value_down, value_up)
    if value < 0 and value2 < 0:
        print("predict: ", value, value2, value_down)
    if value > 0 and value2 > 0:
        print("predict: ", value, value2, value_up)
price = get_current_price(symbol)
print(price, value_adj)
if merton:
    print("Merton: {} {} {}".format(merton["dd"],
                                 round(float(merton["pd"]), 2),
                                   merton["snp_rating"]))

-23.17627 -28.655045 -73.81115 1 -1
predict:  -23.17627 -28.655045 1
52.77 -73.81115
Merton: 0.945480266 0.17 


In [28]:
symbol = "ARGO"
merton = merton_model.get(symbol)
data = mapper_logic.get_dcf_data(symbol)
data_adj = mapper_logic.get_dcf_data_adj(symbol)
if data:
    value = mapper_logic.predict('dcf', data)
    value2 = mapper_logic.predict('dcf_put', data)
    value_adj = mapper_logic.predict('dcf_adj', data_adj)
    value_up = mapper_logic.predict('dcf_binary_up', data)
    value_down = mapper_logic.predict('dcf_binary_down', data)
    print (value, value2, value_adj, value_down, value_up)
    if value < 0 and value2 < 0:
        print("predict: ", value, value2, value_down)
    if value > 0 and value2 > 0:
        print("predict: ", value, value2, value_up)
price = get_current_price(symbol)
print(price, value_adj)
if merton:
    print("Merton: {} {} {}".format(merton["dd"],
                                 round(float(merton["pd"]), 2),
                                   merton["snp_rating"]))

-49.052353 1.4902306 -29.263216 -1 -1
33.6 -29.263216
Merton: 171.6316269 0.0 N/A
